<a href="https://colab.research.google.com/github/Kiyoshi-Tashiro/material-design-icons/blob/master/qulacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install qulacs

In [2]:
pip install qulacs


     |████████████████████████████████| 450kB 10.1MB/s 


# Run Bernstein–Vazirani algorithm

In [23]:
from qulacs import QuantumState
from qulacs.gate import X, Z, H, DenseMatrix
import numpy as np


def show_quantum_state(state, eps = 1e-10, round_digit=3):
  vector = state.get_vector()
  state_str = ""
  qubit_count = int( np.log2(len(vector)+eps))
  binary_format = "{" + ":0{}b".format(qubit_count) + "}"
  for ind in range(len(vector)):
    if abs(vector[ind]) > 1e-10:
      if len(state_str) > 0:
        state_str += " + "
      state_str += ("{} |" + binary_format + ">").format(round(vector[ind],round_digit ),ind)
  print(state_str)

num_bits = 3

import random
secret_a = [random.randint(0, 1) for x in range(num_bits)]

def f(x):
  bstr = ("{" + ":0{}b".format(num_bits) + "}").format(x)
  lx = [int(c) for c in bstr]

  sum = 0
  for i in range(num_bits):
    sum += lx[i] * secret_a[i]
  return sum % 2

cu_mat = np.zeros((2 ** (num_bits + 1), 2 ** (num_bits + 1)), np.int8)
for x in range(2 ** num_bits):
  crow = x * 2 + f(x)
  cu_mat[x*2][crow] = 1
  cu_mat[x*2+1][crow // 2 * 4 + 1 - crow] = 1

if num_bits < 6:
  print("CU Gate Matrix:")
  for i in range(2 ** (num_bits + 1)):
    print(''.join([str(b) for b in cu_mat[i]]))

state = QuantumState(num_bits + 1)
state.set_computational_basis(0)

print("\nInitial State:")
show_quantum_state(state)

for i in range(1, num_bits + 1):
  h_gate = H(i)
  h_gate.update_quantum_state(state)

print("\nAfter H Gate:")
show_quantum_state(state)

cu_gate = DenseMatrix(tuple(range(num_bits + 1)), cu_mat)
cu_gate.update_quantum_state(state)

print("\nAfter CU Gate:")
show_quantum_state(state)

z_gate = Z(0)
z_gate.update_quantum_state(state)

print("\nAfter Z Gate:")
show_quantum_state(state)

cu_gate = DenseMatrix(tuple(range(num_bits + 1)), cu_mat)
cu_gate.update_quantum_state(state)

print("\nAfter CU Gate:")
show_quantum_state(state)

for i in range(1, num_bits + 1):
  h_gate = H(i)
  h_gate.update_quantum_state(state)

print("\nAfter H Gate:")
show_quantum_state(state)

print("\nSecret A was:")
print(secret_a)


del state

CU Gate Matrix:
1000000000000000
0100000000000000
0010000000000000
0001000000000000
0000010000000000
0000100000000000
0000000100000000
0000001000000000
0000000001000000
0000000010000000
0000000000010000
0000000000100000
0000000000001000
0000000000000100
0000000000000010
0000000000000001

Initial State:
(1+0j) |0000>

After H Gate:
(0.354+0j) |0000> + (0.354+0j) |0010> + (0.354+0j) |0100> + (0.354+0j) |0110> + (0.354+0j) |1000> + (0.354+0j) |1010> + (0.354+0j) |1100> + (0.354+0j) |1110>

After CU Gate:
(0.354+0j) |0000> + (0.354+0j) |0010> + (0.354+0j) |0101> + (0.354+0j) |0111> + (0.354+0j) |1001> + (0.354+0j) |1011> + (0.354+0j) |1100> + (0.354+0j) |1110>

After Z Gate:
(0.354+0j) |0000> + (0.354+0j) |0010> + (-0.354-0j) |0101> + (-0.354-0j) |0111> + (-0.354-0j) |1001> + (-0.354-0j) |1011> + (0.354+0j) |1100> + (0.354+0j) |1110>

After CU Gate:
(0.354+0j) |0000> + (0.354+0j) |0010> + (-0.354+0j) |0100> + (-0.354+0j) |0110> + (-0.354+0j) |1000> + (-0.354+0j) |1010> + (0.354+0j) |1100> 

# Run Deutsch–Jozsa algorithm

In [ ]:
from qulacs import QuantumState
from qulacs.gate import X, Z, H, DenseMatrix
import numpy as np


def show_quantum_state(state, eps = 1e-10, round_digit=3):
  vector = state.get_vector()
  state_str = ""
  qubit_count = int( np.log2(len(vector)+eps))
  binary_format = "{" + ":0{}b".format(qubit_count) + "}"
  for ind in range(len(vector)):
    if abs(vector[ind]) > 1e-10:
      if len(state_str) > 0:
        state_str += " + "
      state_str += ("{} |" + binary_format + ">").format(round(vector[ind],round_digit ),ind)
  print(state_str)

num_bits = 10

import random

if random.randint(0, 1):
  y = random.randint(0, 1)
  def f(x):
    return y
  fx_type = 'constant'
else:
  y_dict = {}
  for i in range(2 ** num_bits):
    y_dict[i] = 0
  for i in range(2 ** (num_bits - 1)):
    while True:
      p = random.randint(0, 2 ** num_bits - 1)
      if y_dict[p] == 0:
        y_dict[p] = 1
        break
  def f(x):
    return y_dict[x]
  fx_type = 'balanced'
    

cu_mat = np.zeros((2 ** (num_bits + 1), 2 ** (num_bits + 1)), np.int8)
for x in range(2 ** num_bits):
  crow = x * 2 + f(x)
  cu_mat[x*2][crow] = 1
  cu_mat[x*2+1][crow // 2 * 4 + 1 - crow] = 1

if num_bits < 6:
  print("CU Gate Matrix:")
  for i in range(2 ** (num_bits + 1)):
    print(''.join([str(b) for b in cu_mat[i]]))

state = QuantumState(num_bits + 1)
state.set_computational_basis(0)

print("\nInitial State:")
show_quantum_state(state)

for i in range(1, num_bits + 1):
  h_gate = H(i)
  h_gate.update_quantum_state(state)

print("\nAfter H Gate:")
show_quantum_state(state)

cu_gate = DenseMatrix(tuple(range(num_bits + 1)), cu_mat)
cu_gate.update_quantum_state(state)

print("\nAfter CU Gate:")
show_quantum_state(state)

z_gate = Z(0)
z_gate.update_quantum_state(state)

print("\nAfter Z Gate:")
show_quantum_state(state)

cu_gate = DenseMatrix(tuple(range(num_bits + 1)), cu_mat)
cu_gate.update_quantum_state(state)

print("\nAfter CU Gate:")
show_quantum_state(state)

for i in range(1, num_bits + 1):
  h_gate = H(i)
  h_gate.update_quantum_state(state)

print("\nAfter H Gate:")
show_quantum_state(state)

print("\ntype of f(x) was:")
print(fx_type)


del state


Initial State:
(1+0j) |00000000000>

After H Gate:
(0.031+0j) |00000000000> + (0.031+0j) |00000000010> + (0.031+0j) |00000000100> + (0.031+0j) |00000000110> + (0.031+0j) |00000001000> + (0.031+0j) |00000001010> + (0.031+0j) |00000001100> + (0.031+0j) |00000001110> + (0.031+0j) |00000010000> + (0.031+0j) |00000010010> + (0.031+0j) |00000010100> + (0.031+0j) |00000010110> + (0.031+0j) |00000011000> + (0.031+0j) |00000011010> + (0.031+0j) |00000011100> + (0.031+0j) |00000011110> + (0.031+0j) |00000100000> + (0.031+0j) |00000100010> + (0.031+0j) |00000100100> + (0.031+0j) |00000100110> + (0.031+0j) |00000101000> + (0.031+0j) |00000101010> + (0.031+0j) |00000101100> + (0.031+0j) |00000101110> + (0.031+0j) |00000110000> + (0.031+0j) |00000110010> + (0.031+0j) |00000110100> + (0.031+0j) |00000110110> + (0.031+0j) |00000111000> + (0.031+0j) |00000111010> + (0.031+0j) |00000111100> + (0.031+0j) |00000111110> + (0.031+0j) |00001000000> + (0.031+0j) |00001000010> + (0.031+0j) |00001000100> + (0.

In [33]:
import numpy as np
import random

G1 = np.array((
    (1., 0., 0., 0.),
    (0., 1., 0., 0.),
    (0., 0., 1., 0.),
    (0., 0., 0., 1.),
    (0., 1., 1., 1.),
    (1., 0., 1., 1.),
    (1., 1., 0., 1.)
))

H1 = np.array((
    (0., 1., 1., 1., 1., 0., 0),
    (1., 0., 1., 1., 0., 1., 0.),
    (1., 1., 0., 1., 0., 0., 1.)
))

G2 = H1.T
H2 = G1.T

# np.dot(H1, G2) % 2

x0 = np.array((0., 0., 0., 0., 0., 0., 0.))
x1 = np.array((1., 1., 1., 1., 1., 1., 1.))

# print(np.add(H1[0], H1[1], H1[2]) % 2)
L0 = np.array([(np.sum(np.array([H1[x] * (c // (2 ** x) % 2) for x in range(3)]), axis=0) + x0) % 2 for c in range(2 ** 3)])
print(L0)
L1 = np.array([(np.sum(np.array([H1[x] * (c // (2 ** x) % 2) for x in range(3)]), axis=0) + x1) % 2 for c in range(2 ** 3)])
print(L1)

L0n = [int(''.join(['1' if v > .5 else '0' for v in L0[i]]), 2) for i in range(8)]
print(L0n)
L1n = [int(''.join(['1' if v > .5 else '0' for v in L1[i]]), 2) for i in range(8)]
print(L1n)

G2n = [int(''.join(['1' if v > .5 else '0' for v in G2[i]]), 2) for i in range(7)]
print(G2n)

from qulacs import QuantumState, Observable
from qulacs.gate import RandomUnitary, Pauli, CNOT

def show_quantum_state(state, eps = 1e-10, round_digit=3):
  vector = state.get_vector()
  state_str = ""
  qubit_count = int( np.log2(len(vector)+eps))
  binary_format = "{" + ":0{}b".format(qubit_count) + "}"
  for ind in range(len(vector)):
    if abs(vector[ind]) > 1e-10:
      if len(state_str) > 0:
        state_str += " + "
      state_str += ("{} |" + binary_format + ">").format(round(vector[ind],round_digit ),ind)
  print(state_str)

org_bit = 0

"""
encode_circuit = QuantumCircuit(7 + 3)
for i in range(8):
  if 
  encode_circuit.add_CNOT_gate(0, ind)
"""

state = QuantumState(7 + 3)
# state.set_Haar_random_state(0)
state.set_computational_basis(org_bit)
RandomUnitary([0]).update_quantum_state(state)
print("oroginal qubit")
show_quantum_state(state)

"""
cu_mat = np.zeros((2 ** 7, 2 ** 7), np.int8)
for x in range(2 ** (7 - 1)):
  for y in L0n:
    cu_mat[x*2][y] = 1.
  for y in L1n:
    cu_mat[x*2+1][y] = 1.

print(cu_mat)
cu_gate = DenseMatrix(tuple(range(7)), cu_mat)
cu_gate.update_quantum_state(state)
"""

vec = np.zeros(2 ** (7 + 3), dtype=np.complex)
for i in L0n:
  vec[i] = state.get_vector()[0] / ((2 ** 3) ** .5)
for i in L1n:
  vec[i] = state.get_vector()[1] / ((2 ** 3) ** .5)

# print(vec)

estate = QuantumState(7 + 3)
estate.load(vec)

print("encoded")
# print(state.get_vector())
show_quantum_state(estate)

noised_bit = random.randint(0, 6)
Pauli([noised_bit], [1]).update_quantum_state(estate)

print("noised")
print(noised_bit)
show_quantum_state(estate)

for h in range(3):
  for i, b in enumerate(H1[h]):
    if b > .5:
      CNOT(6 - i, 9 -  h).update_quantum_state(estate)

print("syndrome")
show_quantum_state(estate)

vec = estate.get_vector()
syn_dic = {}
for n, val in enumerate(vec):
  if abs(val) > 1e-10:
    syn = n // (2 ** 7)
    syn_dic[syn] = 1
print(syn_dic)
assert len(syn_dic) == 1

for k in syn_dic.keys():
  syn = k
  break

print(bin(syn))

for i, v in enumerate(G2n):
  if syn == v:
    print("noised bit:", 6 - i)
    break

"""
obs = Observable(7 + 3)
obs.add_operator(1.0, "Z 0")

value = obs.get_expectation_value(estate)
print(value)
"""

# for h in range(7):



[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0.]
 [1. 0. 1. 1. 0. 1. 0.]
 [1. 1. 0. 0. 1. 1. 0.]
 [1. 1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1.]
 [0. 1. 1. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 1. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]
[0, 60, 90, 102, 105, 85, 51, 15]
[127, 67, 37, 25, 22, 42, 76, 112]
[3, 5, 6, 7, 4, 2, 1]
oroginal qubit
(-0.431+0.41j) |0000000000> + (-0.691-0.41j) |0000000001>
encoded
(-0.152+0.145j) |0000000000> + (-0.152+0.145j) |0000001111> + (-0.244-0.145j) |0000010110> + (-0.244-0.145j) |0000011001> + (-0.244-0.145j) |0000100101> + (-0.244-0.145j) |0000101010> + (-0.152+0.145j) |0000110011> + (-0.152+0.145j) |0000111100> + (-0.244-0.145j) |0001000011> + (-0.244-0.145j) |0001001100> + (-0.152+0.145j) |0001010101> + (-0.152+0.145j) |0001011010> + (-0.152+0.145j) |0001100110> + (-0.152+0.145j) |0001101001> + (-0.244-

'\nobs = Observable(7 + 3)\nobs.add_operator(1.0, "Z 0")\n\nvalue = obs.get_expectation_value(estate)\nprint(value)\n'